# 최적의 선수 선택
1. 급여를 1억 1200만달러 넘지 않는 조건
2. 클러스터별 필요 인원을 설정한 후 선수 선택
3. 선택한 선수들의 평균 평점이 가장 높은 경우의 수를 탐색

In [89]:
import pandas as pd
# 경로 설정
df_final=pd.read_csv('/content/final_salary_cluster.csv')

In [90]:
import pandas as pd

# df_final을 사용한다고 가정 (선수 데이터 포함)
df = df_final.copy()

# 총 급여 한도 설정
salary_cap = 112000000

# 각 클러스터별로 필요한 인원 수 설정
cluster_requirements = {
    0: 1,  # 클러스터 0: 1명
    1: 3,  # 클러스터 1: 3명
    2: 1,  # 클러스터 2: 1명
    3: 2,  # 클러스터 3: 2명
    4: 1,  # 클러스터 4: 1명
    5: 1   # 클러스터 5: 1명
}

selected_players = []
for cluster, num_players in cluster_requirements.items():
    top_players = df[df['Cluster'] == cluster].nlargest(num_players, '전체적인 평점')
    selected_players.extend(top_players[['PLAYER', 'Salary', '전체적인 평점', 'Cluster']].to_dict('records'))

excluded_players = [p['PLAYER'] for p in selected_players]



In [91]:
selected_players

[{'PLAYER': 'Jrue Holiday',
  'Salary': 34319520.0,
  '전체적인 평점': 8.322794916831938,
  'Cluster': 0},
 {'PLAYER': 'Franz Wagner',
  'Salary': 5258280.0,
  '전체적인 평점': 8.155521414745108,
  'Cluster': 1},
 {'PLAYER': 'Terry Rozier',
  'Salary': 21486316.0,
  '전체적인 평점': 7.634911279767859,
  'Cluster': 1},
 {'PLAYER': 'Dejounte Murray',
  'Salary': 17071120.0,
  '전체적인 평점': 7.5698869944078115,
  'Cluster': 1},
 {'PLAYER': 'Joel Embiid',
  'Salary': 33616770.0,
  '전체적인 평점': 13.005239828767696,
  'Cluster': 2},
 {'PLAYER': 'Giannis Antetokounmpo',
  'Salary': 42492492.0,
  '전체적인 평점': 15.068057338989538,
  'Cluster': 3},
 {'PLAYER': 'Jayson Tatum',
  'Salary': 30351780.0,
  '전체적인 평점': 13.797792368776374,
  'Cluster': 3},
 {'PLAYER': 'Darius Garland',
  'Salary': 8920794.0,
  '전체적인 평점': 9.757136447969048,
  'Cluster': 4},
 {'PLAYER': 'Jamal Murray',
  'Salary': 31650600.0,
  '전체적인 평점': 8.4410867274808,
  'Cluster': 5}]

In [92]:
# 3. 팀의 총 급여 계산
total_salary = sum(player['Salary'] for player in selected_players)
total_salary

225167672.0

In [93]:
# 2. 가장 비싼 선수를 대체하는 함수
def replace_most_expensive_player(team, df, excluded_players):
    # 2-1. 가장 비싼 선수 찾기
    most_expensive_player = max(team, key=lambda x: x['Salary'])  # Salary 기준
    cluster = most_expensive_player['Cluster']  # 해당 선수의 클러스터

    # 2-2. 해당 클러스터에서 남은 후보들 중에서 다음으로 평점이 높은 선수 선택
    # excluded_players에 포함되지 않은 선수들만 후보에 추가
    candidates = df[(df['Cluster'] == cluster) &
                    (~df['PLAYER'].isin([p['PLAYER'] for p in team])) &
                    (~df['PLAYER'].isin(excluded_players))]
    if candidates.empty:
        return team  # 대체할 선수가 없는 경우 종료

    # 가장 높은 평점을 가진 선수 선택
    replacement = candidates.nlargest(1, '전체적인 평점').iloc[0].to_dict()

    # 2-3. 팀에서 가장 비싼 선수를 대체
    new_team = [p for p in team if p != most_expensive_player]
    new_team.append(replacement)

    return new_team, replacement['PLAYER']  # 교체된 선수 이름 반환



In [94]:
while total_salary > salary_cap:
    selected_players, replaced_player = replace_most_expensive_player(selected_players, df, excluded_players)
    excluded_players.append(replaced_player)  # 제외 선수 목록에 추가
    total_salary = sum(player['Salary'] for player in selected_players)

In [97]:
# 5. 최종 팀의 총 평점과 평균 평점 계산
total_rate = sum(player['전체적인 평점'] for player in selected_players)
rate_avg = total_rate / len(selected_players)

# 6. 최종 팀 데이터프레임으로 변환 및 출력
final_team_df = pd.DataFrame(selected_players)
print("선택된 팀:")
print(final_team_df)
print(f"평균 평점: {rate_avg}, 총 급여: {total_salary}")


선택된 팀:
             PLAYER      Salary    전체적인 평점  Cluster   SEASON TEAM   MIN
0      Franz Wagner   5258280.0   8.155521        1      NaN  NaN   NaN
1   Dejounte Murray  17071120.0   7.569887        1      NaN  NaN   NaN
2    Darius Garland   8920794.0   9.757136        4      NaN  NaN   NaN
3  Kevin Porter Jr.   3200000.0   7.747722        0  2022-23  HOU  34.3
4       Evan Mobley   8478720.0   7.082348        5  2022-23  CLE  34.4
5         Ja Morant  12119440.0  10.597250        2  2022-23  MEM  31.9
6   Anthony Edwards  10733400.0  10.039151        3  2022-23  MIN  36.0
7       Jalen Green   9441840.0   9.396637        3  2022-23  HOU  34.2
8     Jarrett Allen  20000000.0   6.307101        1  2022-23  CLE  32.6
평균 평점: 8.516972507458323, 총 급여: 95223594.0
